In [1]:
import numpy as np
import pandas as pd
pd.get_option("display.max_columns")
pd.set_option('display.max_columns', 300)
pd.get_option("display.max_rows")
pd.set_option('display.max_rows', 300)

import matplotlib.pyplot as plt
%matplotlib inline

import os
from os.path import join as opj
import gc

INPUT_PATH = '../../input/feedback-prize-english-language-learning/'

train_df = pd.read_csv(opj(INPUT_PATH, 'train.csv'))
test_df = pd.read_csv(opj(INPUT_PATH, 'test.csv'))
sub_df = pd.read_csv(opj(INPUT_PATH, 'sample_submission.csv'))

print('train_df.shape = ', train_df.shape)
print('test_df.shape = ', test_df.shape)
print('sub_df.shape = ', sub_df.shape)

train_df.shape =  (3911, 8)
test_df.shape =  (3, 2)
sub_df.shape =  (3, 7)


In [3]:
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [5]:
model_name = 'microsoft/deberta-v3-base'

from transformers.models.deberta_v2 import DebertaV2TokenizerFast
tokenizer = DebertaV2TokenizerFast.from_pretrained(model_name, trim_offsets=False)
special_tokens_dict = {'additional_special_tokens': ['\n\n']}
_ = tokenizer.add_special_tokens(special_tokens_dict)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
from autocorrect import Speller
spell = Speller()
spell("I'm not sleapy and tehre is no place I'm giong to.")
#"I'm not sleepy and there is no place I'm going to."

"I'm not sleepy and there is no place I'm going to."

In [48]:
from difflib import SequenceMatcher

df = train_df[train_df['vocabulary']==2.0].reset_index(drop=True)

for idx in range(100):
    text = df.loc[idx,'full_text'] 
    s = SequenceMatcher(None, text, spell(text))
    print('idx={}, ratio={}'.format(idx,s.ratio()))

idx=0, ratio=0.9996791786974655
idx=1, ratio=0.4663710273466371
idx=2, ratio=0.9947381608619393
idx=3, ratio=0.9908008658008658
idx=4, ratio=0.9308615049073065
idx=5, ratio=0.6450247000705717
idx=6, ratio=0.9987966305655837
idx=7, ratio=0.9968701095461658
idx=8, ratio=0.8787226408324363
idx=9, ratio=0.9446529080675422
idx=10, ratio=0.9228709159078736
idx=11, ratio=0.9960899315738025
idx=12, ratio=0.9978632478632479
idx=13, ratio=0.99581589958159
idx=14, ratio=0.9007858546168959
idx=15, ratio=0.9979575805184603
idx=16, ratio=0.9990497307570478
idx=17, ratio=0.9941438275942843
idx=18, ratio=0.9950450450450451
idx=19, ratio=0.9565667011375388
idx=20, ratio=0.9876304599922691
idx=21, ratio=0.9920870425321464
idx=22, ratio=0.9981606376456161
idx=23, ratio=1.0
idx=24, ratio=0.9971799210377891
idx=25, ratio=1.0
idx=26, ratio=0.9991428081604663
idx=27, ratio=0.8954671600370028
idx=28, ratio=0.9979729729729729
idx=29, ratio=0.9979716024340771
idx=30, ratio=0.7790235081374322
idx=31, ratio=1.0
i

In [49]:
df = train_df[train_df['vocabulary']==2.0].reset_index(drop=True)

idx = 2
text = df.loc[idx,'full_text'] 

print(text)
print('\n' + '*'*20 + '\n') 
print(spell(text))

If you image the world student stay home take online class? that be great for student are most sake can not go to school but online class is big help for sake or in the hospital. they are people hoe have problem like have cancers or harder dazes is not harm full anyone student in school. but some student they say their is no problem to not to come school if you sake or not you have come to school. but they are range because they do not hate to go school they body is not strong handily jar mes in and get sake al the time and school allowing ha very to can take online class but they have pay. but student way not free because student hoe are sake pay all the class and they can not play any sport, your student can come to great full of your parent.

do have thing in their power sand ache one to school .

very I was 5 year old choke cold toke my my parent,

I do not wont to go school star cry Generic_Name hurt van my gram father hired what i say come to and say stop cry eat your maidens and

In [38]:
df = train_df[train_df['vocabulary']==2.0].reset_index(drop=True)


#idx = 4
for idx in range(20):
    text = df.loc[idx,'full_text']
    gt_voc = df.loc[idx,'vocabulary']
    #print('vocabulary = ', gt_voc)
    #
    #print(text)
    print('*'*20)
    print(''.join(tokenizer.tokenize(text, add_special_tokens=True)))
    print('')

********************
[CLS]▁When,▁the▁people▁decide▁to▁have▁a▁good▁posture,▁in▁they▁live▁of▁thing▁was▁be▁better.▁Even▁though▁some▁people▁have▁a▁negative▁attitude▁in▁their▁live,▁all▁the▁thing▁was▁bad▁and▁not▁want▁doing▁noting▁for▁fair▁to▁make▁mistakes,nevertheless▁people▁has▁a▁good▁attitude▁always,▁have▁a▁good▁attitude▁see▁the▁live▁of▁the▁other▁way,for▁example▁when▁give▁you▁a▁so▁bad▁news▁but▁you▁take▁a▁good▁posture▁and▁said▁is▁okay▁everything▁was▁good,and▁take▁a▁good▁attitude▁in▁the▁school.

▁If▁the▁people▁will▁have▁a▁good▁posture▁in▁all▁position▁of▁their▁live,▁not▁have▁so▁much▁preocupation▁they▁was▁be▁more▁happy.▁when▁you▁have▁a▁big▁text▁in▁the▁school▁although,

▁you▁never▁can▁pass▁this▁text▁but▁you▁have▁a▁positive▁attitude,▁of▁one▁you▁console▁will▁call▁you,▁and▁said▁you▁pass▁the▁text.▁This▁is▁important▁because▁the▁people▁believe▁in▁will▁pass▁and▁when▁they▁pass▁was▁feel▁so▁good,▁because▁they▁this▁text▁was▁so▁hart,▁but▁never▁to▁surrender.

▁when▁one▁person▁go▁to▁the▁hospital▁and▁the▁doct